In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import json

# Read Data

In [3]:
metadata = pd.read_csv('./data/metadata.csv')
all_india = metadata[metadata['Division']=='India']

for i,row in all_india.iterrows():
    all_india.loc[i, ["Local_lat"]] = float(row["Local_lat"])
    
north_indian_states = {
       'Chhattisgarh',
       'Maharashtra', 'Malwa',
       'Chota Nagpur Plateau',
       'Bihar/ Uttar Pradesh', 'Uttar Pradesh', 'Varanasi, Uttar Pradesh',
       'Rajasthan', 'Gujarat',
       'Gujarat/ Madhya Pradesh/ Maharashtra/ Rajasthan'
}

north_india = pd.DataFrame()
south_india = pd.DataFrame()

for i,row in all_india.iterrows():
    if row["Area/Kingdom"] in north_indian_states:
        north_india = north_india.append(all_india.loc[i])
    else:
        south_india = south_india.append(row)
        
north_india = north_india.rename(columns={'C-id':'canto_coding_id'})
south_india = south_india.rename(columns={'C-id':'canto_coding_id'})

north_india.to_csv('./data/north-india.csv')
south_india.to_csv('./data/south-india.csv')

/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [4]:
classfication_metadata = pd.read_csv('./data/classification_metadata.csv',encoding='latin1')
classfication_metadata[classfication_metadata['Culture']=='Dhurwa Gond']['Lang_Family'].values[0]
language_map = {
    'Dravidian, Central Dravidian':'Dravidian',
    'Dravidian, South Dravidian, Gondi, Koya': 'Dravidian',
    'Dravidian, South Dravidian, Gondi' :'Dravidian',
    'Dravidian, South Dravidian': 'Dravidian',
    'Dravidian, North Dravidian, Kurux-Malto, Kurux':'Dravidian',
    'Indo-European, Indo-Iranian, Indo-Aryan, Western Hindi, Hindustani': 'Indo-European',
    'Austroasiatic, Mundaic, North Munda': 'Austroasiatic',
    'Austroasiatic, Mundaic, Mundaric, Ho-Mundari':'Austroasiatic',
    'Austroasiatic, Mundaic, North Munda, Mundaric':'Austroasiatic',
    'Austroasiatic, Mundaic, North Munda, Kherwarian':'Austroasiatic',
    'Indo-European, Indo-Iranian, Indo-Aryan, Bihari, Western Magadhan, Bhojpuric':'Indo-European',
    'Indo-European, Indo-Iranian, Indo-Aryan, Gujarati-Rajasthani':'Indo-European',
    'Indo-European, Indo-Iranian, Indo-Aryan, Gujarati-Rajasthani, Gujaratic':'Indo-European',
    'Indo-European, Indo-Iranian, Indo-Aryan, Subcontinental Central Indo-Aryan':'Indo-European',
    'Indo-European, Indo-Iranian, Indo-Aryan, Rajasthani':'Indo-European',
    'Dravidian, Central Dravidian, Kolami-Naiki':'Dravidian',
    'Indo-European, Indo-Iranian, Indo-Aryan, Marathic':'Indo-European',
    'Dravidian, South Dravidian, Tamil-Kannada, Tamil-Malayalam, Tamiloid':'Dravidian',
    'Dravidian, South Dravidian, Teluguic':'Dravidian',
    'Dravidian, South Dravidian, Gondi':'Dravidian',
    'Dravidian, Parji-Ollari-Gadaba, Ollari-Gadaba':'Dravidian',
    'Austroasiatic, Mundaic, Sora-Juray-Gorum':'Austroasiatic',
    'Dravidian, South Dravidian, Konda-Kui':'Dravidian',
    'Austroasiatic, Mundaic, South Munda, Gutob-Remo':'Austroasiatic',
    'Indo-European, Indo-Iranian, Indo-Aryan, Oriya-Gauda-Kamrupa, Macro-Oriya':'Indo-European',
    'Indo-European, Indo-Iranian, Indo-Aryan, Indo-Aryan Eastern Zone':'Indo-European',
    'Indo-European, Italic, Latinic, Romance, Western Romance, Galician Romance, Macro-Portuguese':'Indo-European',
    'Dravidian, South Dravidian, Tamil-Kannada, Tamil-Malayalam, Malayamoid':'Dravidian',
    'Dravidian, South Dravidian, Tamil-Kannada, Tamil-Toda':'Dravidian',
}

In [5]:
north_india = pd.read_csv('./data/north-india.csv')
south_india = pd.read_csv('./data/south-india.csv')
metadata = pd.read_csv('./data/metadata.csv')
with open('./output/output.json') as json_file:
    codebook = json.load(json_file)

In [6]:
print("Total number of tracks is",len(north_india)+len(south_india))

Total number of tracks is 208


## Draw from Google Sheets

In [7]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# Reference: https://developers.google.com/sheets/api/quickstart/python
def read_google_sheets(SPREADSHEET_ID, RANGE_NAME, HEADER_RANGE):
    creds = None
    # autogenerated
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                range=RANGE_NAME).execute()
    
    header = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                               range=HEADER_RANGE).execute()
    
    header_values = header.get('values', [])
    values = result.get('values', [])
    
    return values, header_values

SPREADSHEET_ID = '1AjynK9mMQTw58B_B8b_ZIip3fyUm-aoV7Pp21HziBb0'
RANGE_NAME = 'canto_codings!A2:AT'
HEADER_RANGE = 'canto_codings!A1:AT1'

data, header = read_google_sheets(SPREADSHEET_ID, RANGE_NAME, HEADER_RANGE)

canto_codings = pd.DataFrame(data, columns = header[0])

## Utility Functions

In [8]:
def find_canto_features(canto_coding_id):
    for i,row in canto_codings.iterrows():
        if(str(canto_coding_id) == row['canto_coding_id']):
            return row

In [9]:
def get_canto_metadata(canto_coding_id):
    for i, row in metadata.iterrows():
        if(str(canto_coding_id) == row['C-id']):
            return row

In [10]:
def get_display_code(line, binary_code):
    line = codebook['line_'+str(line)]
    for encoding in line:
        if encoding['code'] == str(binary_code):
            return encoding['display_code']

In [11]:
def get_language_family(culture):
    indian_metadata = classfication_metadata[classfication_metadata['Culture']==culture]
    try:
        language = indian_metadata['Lang_Family'].values[0]
        people = indian_metadata['People'].values[0]
        if "(Ethnic Peoples)" in people:
            people = "Tribal"
        else:
            people = "Non-Tribal"
    except IndexError as e:
        return "Dravidian", "Tribal"
    return language_map[language], people

# Prepare Output Data Structure

In [155]:
columns = ["canto_coding_id", "region", "division","subregion", "area_kingdom", "culture", "language", "people","song","lat", "lng"]

for i in range(37):
    columns.append("cv_"+str(i+1))
    
north_india_full = pd.DataFrame(columns=columns)
south_india_full = pd.DataFrame(columns=columns)

# Connect Everything

In [156]:
def main(input_matrix, output_matrix):
    for i, row in input_matrix.iterrows():
        canto_coding_id = row['canto_coding_id']
        culture = row['Culture']
        meta = get_canto_metadata(int(canto_coding_id))
        lat = meta['Local_lat']
        lng = meta['Local_long']
        region = meta['Region']
        division = meta['Division']
        subregion = meta['Subregion']
        area = meta['Area/Kingdom']
        song = meta['Song']
        language,people = get_language_family(culture)
        canto = find_canto_features(canto_coding_id)
        canto_data = []
        for i in range(37):
            canto_data.append(get_display_code(i+1, canto["cv_"+str(i+1)]))
        new_row = pd.DataFrame([[
            canto_coding_id,
            region,
            division,
            subregion,
            area,
            culture,
            language,
            people,
            song,
            lat,
            lng,
            canto_data[0],
            canto_data[1],
            canto_data[2],
            canto_data[3],
            canto_data[4],
            canto_data[5],
            canto_data[6],
            canto_data[7],
            canto_data[8],
            canto_data[9],
            canto_data[10],
            canto_data[11],
            canto_data[12],
            canto_data[13],
            canto_data[14],
            canto_data[15],
            canto_data[16],
            canto_data[17],
            canto_data[18],
            canto_data[19],
            canto_data[20],
            canto_data[21],
            canto_data[22],
            canto_data[23],
            canto_data[24],
            canto_data[25],
            canto_data[26],
            canto_data[27],
            canto_data[28],
            canto_data[29],
            canto_data[30],
            canto_data[31],
            canto_data[32],
            canto_data[33],
            canto_data[34],
            canto_data[35],
            canto_data[36],
        ]], columns = columns)
        output_matrix = output_matrix.append(new_row)
    return output_matrix

## Run data conversion. This will take a while

In [157]:
north_india_full = main(north_india, north_india_full)

In [158]:
south_india_full = main(south_india, south_india_full)

## Write data to disk

In [159]:
north_india_full.to_csv('./data/north_india_full.csv', index=False)
south_india_full.to_csv('./data/south_india_full.csv', index=False)

In [160]:
region = []
for i in range(len(north_india_full)):
    region.append("North")
for i in range(len(south_india_full)):
    region.append("South")

frames = [north_india_full, south_india_full]
all_india_full = pd.concat(frames)
all_india_full["region"] = region
all_india_full = all_india_full[all_india_full["culture"]!="Portuguese Goa"]
all_india_full.to_csv('./data/all_india_full.csv', index=False)

In [40]:
all_india_full = pd.read_csv('./data/all_india_full.csv')
all_india_full['language'].value_counts()

Dravidian        101
Indo-European     55
Austroasiatic     51
Name: language, dtype: int64

In [41]:
all_india_full['people'].value_counts()

Tribal        150
Non-Tribal     57
Name: people, dtype: int64

In [42]:
c = 0
for i, row in all_india_full.iterrows():
    if row['language'] != 'Austroasiatic' and row['people']=='Non-Tribal':
        c+=1
c

47

## Important: Change the file's content so that Tamil People are coded as dravidians!

# Preparing Data for AMOVA for Indian Sample in R

## Removing Cultures With Only One Sample

In [43]:
from scipy.spatial import distance_matrix
import math
import numpy as np

## Samples Dataframe

In [141]:
all_india_full = pd.read_csv('./data/all_india_full.csv')

In [142]:
# This value was missing. 
all_india_full['cv_1'][196] = 2

<ipython-input-142-0518c67386b4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_india_full['cv_1'][196] = 2


In [143]:
all_india_full['Number of Samples'] = all_india_full.groupby('culture')['culture'].transform('count')
no_singles = all_india_full[all_india_full['Number of Samples']!=1]
singles = all_india_full[all_india_full['Number of Samples']==1]
filtered_df = all_india_full[(all_india_full['language']!='Austroasiatic') & (all_india_full['people']!='Tribal')]

filtered_df.to_csv('./data/filtered.csv')

In [144]:
sample_columns = filtered_df['culture'].unique()
sample_columns = np.insert(sample_columns, 0, 'canto_coding_id', axis=0)
samples = pd.DataFrame(columns=sample_columns)
## fill this up with zeros

for i, row in filtered_df.iterrows():
    samples.loc[i, row['culture']] = 1
    samples.loc[i, 'canto_coding_id'] = row["canto_coding_id"]
    
samples = samples.fillna(0)
samples = samples.reset_index()
samples = samples.drop(columns=['index'])

samples.to_csv('./data/samples.csv')

## Structures Dataframe

In [145]:
structure_list = []
for i in range(len(sample_columns)-1):
    structure_list.append(filtered_df[filtered_df['culture']==sample_columns[i+1]]['language'].all())
structure = pd.DataFrame(structure_list, columns = ['language'])
structure = structure['language'].replace('Indo-European', 'Indo-Aryan')
structure.to_csv('./data/structure.csv')

In [146]:
all_india_full['cv_1'].isnull().sum()

0

# Prepare AMOVA Data for Global Sample in R
## Remove cultures with Only One Sample

In [49]:
canto_codings['Number of Samples'] = canto_codings.groupby('Culture')['Culture'].transform('count')
singles = canto_codings[canto_codings['Number of Samples']!=1]
singles.to_csv('./data/global_no_singles.csv')

## Appending regions to the canto_codings dataframe

In [50]:
canto_codings['region'] = metadata['Region']

## Samples Dataframe
### This will take a while to run

In [29]:
sample_columns = singles['Culture'].unique()
sample_columns = np.insert(sample_columns, 0, 'canto_coding_id', axis=0)
samples = pd.DataFrame(columns=sample_columns)
## fill this up with zeros

for i, row in singles.iterrows():
    samples.loc[i, row['Culture']] = 1
    samples.loc[i, 'canto_coding_id'] = row["canto_coding_id"]
samples = samples.fillna(0)
samples.to_csv('./data/global_samples.csv')

In [30]:
## Structures Dataframe

In [31]:
structure_list = []
for i in range(len(sample_columns)-1):
    structure_list.append(canto_codings[canto_codings['Culture']==sample_columns[i+1]]['region'].all())
structure = pd.DataFrame(structure_list, columns = ['region'])
structure.to_csv('./data/global_structure.csv')

In [152]:
filtered_df['language'].value_counts()

Indo-European    28
Dravidian        19
Name: language, dtype: int64